In [1]:
#basic libraries
import pandas as pd
import numpy as np
import datetime

#library to compute % similarity between strings
from difflib import SequenceMatcher
from difflib import get_close_matches

In [2]:
odoo_leads = pd.read_excel('C:/Users/grego/Desktop/i-lunch/Marketing/CRM/leads/leads_odoo.xlsx')

In [3]:
new_leads = pd.read_excel('C:/Users/grego/Desktop/i-lunch/Marketing/CRM/leads/Pistes Global - All in one sheet.xlsx')

In [4]:
new_leads.head(2)

,BDD Source,Nom,Prénom,Civilité,Position_occupée,Email,Téléphone,Fax,URL Linkedin,Linkedin de l'entreprise,...,Ville,Code postal,Adresse,Tranche d'effectif,CA en k€,Site web de l'entreprise,Année de création,SIRET,Source d'acquisition du lead,.
0,Prospects Sept 2020,Virginie,ROGER ROQUE,Madame,Assistante de Direction - Office Manager,virginie.rogerroque@architas.com,NaN,NaN,NaN,NaN,...,Paris et périphérie,NaN,NaN,51-200,NaN,architas.com,NaN,4.148815e+13,NaN,NaN
1,Prospects Sept 2021,Perrine,Fennebresque Teessen,Madame,Office Manager,perrine@upslide.net,+33 1 83 64 18 40,NaN,NaN,NaN,...,Paris et périphérie,NaN,NaN,51-200,NaN,upslide.net,NaN,7.919258e+13,NaN,NaN


In [5]:
odoo_leads.head(2)

,Opportunité,Nom de la société,Rue,Rue 2,Ville,Pays,Site Web,Vendeur,Statut,Nom du contact,Email,Poste occupé,Téléphone,Mobile,Étiquettes,Source,Nombre de salariés,Filtre_odoo,Prenom,Nom
0,EXCEL COURSES_xxx_Nov20,EXCEL COURSES,22 RUE DIEUMEGARD,NaN,SAINT-OUEN-SUR-SEINE,France,http://www.excelcourses.fr,Sebastien Pigeolet,Converti,SERGE GIRARD,se.girard@excelcourses.fr,Directeur Financier,01 41 40 22 80,NaN,False,Acquisition de fichiers externes,50.0,pipe_opportunitesouvertes,SERGE,GIRARD
1,GTIF_xxx_Nov20,GTIF,140 AV PAUL DOUMER,NaN,RUEIL-MALMAISON,France,http://gtif.fr,Maxime Loste,Converti,CÉCILE NONIS,cecile.nonis@gtif.fr,Responsable Ressources Humaines,01 41 42 06 00,NaN,False,Acquisition de fichiers externes,30.0,pipe_opportunitesouvertes,CÉCILE,NONIS


In [6]:
#on retire les lignes en double
odoo_leads = odoo_leads.drop_duplicates()

In [7]:
#tratiement des numeros de tel

def right(s, amount):
    return s[-amount:]

#creation de id phone pour check 4
new_leads["id_tel"] = new_leads["Téléphone"].str.replace(" ","").astype(str)
odoo_leads["id_tel"] = odoo_leads["Téléphone"].str.replace(" ","").astype(str)

new_leads["id_tel"] = new_leads.apply(lambda x: right(x['id_tel'],8),axis =1)
odoo_leads["id_tel"] = odoo_leads.apply(lambda x: right(x['id_tel'],8),axis =1)

In [8]:
new_leads.fillna("Nan",inplace=True)
new_leads.head(2)

,BDD Source,Nom,Prénom,Civilité,Position_occupée,Email,Téléphone,Fax,URL Linkedin,Linkedin de l'entreprise,...,Code postal,Adresse,Tranche d'effectif,CA en k€,Site web de l'entreprise,Année de création,SIRET,Source d'acquisition du lead,.,id_tel
0,Prospects Sept 2020,Virginie,ROGER ROQUE,Madame,Assistante de Direction - Office Manager,virginie.rogerroque@architas.com,Nan,Nan,Nan,Nan,...,Nan,Nan,51-200,Nan,architas.com,Nan,4.14882e+13,Nan,Nan,nan
1,Prospects Sept 2021,Perrine,Fennebresque Teessen,Madame,Office Manager,perrine@upslide.net,+33 1 83 64 18 40,Nan,Nan,Nan,...,Nan,Nan,51-200,Nan,upslide.net,Nan,7.91926e+13,Nan,Nan,83641840


In [9]:
########################################### check 1 #######################################
sim_company_name =  pd.DataFrame(pd.np.empty((len(odoo_leads), len(new_leads)))) 
sim_company_name.columns = new_leads["Entreprise"]
sim_company_name.index = odoo_leads['Nom de la société']

for i in range(0,sim_company_name.shape[0]):
    for j in range(0,sim_company_name.shape[1]):
        try:
            sim = SequenceMatcher(None,sim_company_name.index[i].lower(),sim_company_name.columns[j].lower()).ratio()
        except:
            sim = -1
        sim_company_name.iat[i,j] = sim

sim_result = pd.DataFrame(pd.np.empty((1, len(new_leads))),dtype = np.str) 
sim_result.columns = new_leads["Entreprise"]

for j in range(0,sim_result.shape[1]):
    #sim_company_name_result.iat[0,j] = sim_company_name.loc[sim_company_name.iloc[:, j] == max(sim_company_name.iloc[:, j])].index[0]
    try:
        sim_result.iat[0,j] = sim_company_name[(sim_company_name.iloc[:, j] == max(sim_company_name.iloc[:, j])) & (sim_company_name.iloc[:, j] >0.8)].index[0]
    except:
        sim_result.iat[0,j] = "Pas de correspondances"


<ipython-input-9-de9fd5e8cbdc>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  sim_company_name =  pd.DataFrame(pd.np.empty((len(odoo_leads), len(new_leads))))
<ipython-input-9-de9fd5e8cbdc>:14: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  sim_result = pd.DataFrame(pd.np.empty((1, len(new_leads))),dtype = np.str)


In [9]:
########################################### check 2 #######################################
sim_mail =  pd.DataFrame(pd.np.empty((len(odoo_leads), len(new_leads)))) 
sim_mail.columns = new_leads["Email"]
sim_mail.index = odoo_leads['Email']

for i in range(0,sim_mail.shape[0]):
    print(round((i/len(new_leads)),3) *100, "%")
    for j in range(0,sim_mail.shape[1]):
        try:
            sim = SequenceMatcher(None,sim_mail.index[i].lower(),sim_mail.columns[j].lower()).ratio()
        except:
            sim = -1
        sim_mail.iat[i,j] = sim
        
sim_result_2 = pd.DataFrame(pd.np.empty((1, len(new_leads))),dtype = np.str) 
sim_result_2.columns = new_leads["Entreprise"]

for j in range(0,sim_result_2.shape[1]):
    #sim_company_name_result.iat[0,j] = sim_company_name.loc[sim_company_name.iloc[:, j] == max(sim_company_name.iloc[:, j])].index[0]
    try:
        sim_result_2.iat[0,j] = sim_mail[(sim_mail.iloc[:, j] == max(sim_mail.iloc[:, j])) & (sim_mail.iloc[:, j] >0.9)].index[0]
    except:
        sim_result_2.iat[0,j] = "Pas de correspondances"

<ipython-input-9-660f05eed976>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  sim_mail =  pd.DataFrame(pd.np.empty((len(odoo_leads), len(new_leads))))


0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.0 %
0.1 %
0.1 %
0.1 %
0.1 %
0.1 %
0.1 %
0.1 %
0.1 %
0.1 %
0.1 %
0.1 %
0.1 %
0.2 %
0.2 %
0.2 %
0.2 %
0.2 %
0.2 %
0.2 %
0.2 %
0.2 %
0.2 %
0.2 %
0.2 %
0.3 %
0.3 %
0.3 %
0.3 %
0.3 %
0.3 %
0.3 %
0.3 %
0.3 %
0.3 %
0.3 %
0.4 %
0.4 %
0.4 %
0.4 %
0.4 %
0.4 %
0.4 %
0.4 %
0.4 %
0.4 %
0.4 %
0.4 %
0.5 %
0.5 %
0.5 %
0.5 %
0.5 %
0.5 %
0.5 %
0.5 %
0.5 %
0.5 %
0.5 %
0.6 %
0.6 %
0.6 %
0.6 %
0.6 %
0.6 %
0.6 %
0.6 %
0.6 %
0.6 %
0.6 %
0.6 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.7000000000000001 %
0.8 %
0.8 %
0.8 %
0.8 %
0.8 %
0.8 %
0.8 %
0.8 %
0.8 %
0.8 %
0.8 %
0.8999999999999999 %
0.8999999999999999 %
0.8999999999999999 %
0.8999999999999999 %
0.8999999999999999 %
0.8999999999999999 %
0.8999999999999999 %
0.8999999999999999 %
0.8999999999999999 %
0.8999999999999999 %
0.89999999999999

6.7 %
6.7 %
6.7 %
6.7 %
6.7 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.800000000000001 %
6.9 %
6.9 %
6.9 %
6.9 %
6.9 %
6.9 %
6.9 %
6.9 %
6.9 %
6.9 %
6.9 %
6.9 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.000000000000001 %
7.1 %
7.1 %
7.1 %
7.1 %
7.1 %
7.1 %
7.1 %
7.1 %
7.1 %
7.1 %
7.1 %
7.1 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.199999999999999 %
7.3 %
7.3 %
7.3 %
7.3 %
7.3 %
7.3 %
7.3 %
7.3 %
7.3 %
7.3 %
7.3 %
7.3999999999999995 %
7.3999999999999995 %
7.3999999999999995 %
7.399999999999999

13.700000000000001 %
13.700000000000001 %
13.700000000000001 %
13.8 %
13.8 %
13.8 %
13.8 %
13.8 %
13.8 %
13.8 %
13.8 %
13.8 %
13.8 %
13.8 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
13.900000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.000000000000002 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.099999999999998 %
14.2 %
14.2 %
14.2 %
14.2 %
14.2 %
14.2 %
14.2 %
14.2 %
14.2 %
14.2 %
14.2 %
14.2 %
14.299999999999999 %
14.299999999999999 %

20.4 %
20.4 %
20.4 %
20.4 %
20.4 %
20.4 %
20.4 %
20.4 %
20.4 %
20.4 %
20.5 %
20.5 %
20.5 %
20.5 %
20.5 %
20.5 %
20.5 %
20.5 %
20.5 %
20.5 %
20.5 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.599999999999998 %
20.7 %
20.7 %
20.7 %
20.7 %
20.7 %
20.7 %
20.7 %
20.7 %
20.7 %
20.7 %
20.7 %
20.7 %
20.8 %
20.8 %
20.8 %
20.8 %
20.8 %
20.8 %
20.8 %
20.8 %
20.8 %
20.8 %
20.8 %
20.9 %
20.9 %
20.9 %
20.9 %
20.9 %
20.9 %
20.9 %
20.9 %
20.9 %
20.9 %
20.9 %
20.9 %
21.0 %
21.0 %
21.0 %
21.0 %
21.0 %
21.0 %
21.0 %
21.0 %
21.0 %
21.0 %
21.0 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.099999999999998 %
21.2 %
21.2 %
21.2 %
21.2 %

...

54.400000000000006 %
54.400000000000006 %
54.400000000000006 %
54.400000000000006 %
54.400000000000006 %
54.400000000000006 %
54.400000000000006 %
54.400000000000006 %
54.400000000000006 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.50000000000001 %
54.6 %
54.6 %
54.6 %
54.6 %
54.6 %
54.6 %
54.6 %
54.6 %
54.6 %
54.6 %
54.6 %
54.6 %
54.7 %
54.7 %
54.7 %
54.7 %
54.7 %
54.7 %
54.7 %
54.7 %
54.7 %
54.7 %
54.7 %
54.7 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.800000000000004 %
54.900000000000006 %
54.900000000000006 %
54.900000000000006 %
54.900000000000006 %
54.900000000000006 %
54.900000000000006 %
54.900000000000006 %
54.900000000000006 %
54.900000000000006 %
54.

58.9 %
58.9 %
58.9 %
58.9 %
58.9 %
59.0 %
59.0 %
59.0 %
59.0 %
59.0 %
59.0 %
59.0 %
59.0 %
59.0 %
59.0 %
59.0 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.099999999999994 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.199999999999996 %
59.3 %
59.3 %
59.3 %
59.3 %
59.3 %
59.3 %
59.3 %
59.3 %
59.3 %
59.3 %
59.3 %
59.3 %
59.4 %
59.4 %
59.4 %
59.4 %
59.4 %
59.4 %
59.4 %
59.4 %
59.4 %
59.4 %
59.4 %
59.4 %
59.5 %
59.5 %
59.5 %
59.5 %
59.5 %
59.5 %
59.5 %
59.5 %
59.5 %
59.5 %
59.5 %
59.599999999999994 %
59.599999999999994 %
59.599999999999994 %
59.599999999999994 %
59.599999999999994 %
59.599999999999994 %
59.599999999999994 %
59.5999999999

66.4 %
66.4 %
66.4 %
66.4 %
66.4 %
66.4 %
66.5 %
66.5 %
66.5 %
66.5 %
66.5 %
66.5 %
66.5 %
66.5 %
66.5 %
66.5 %
66.5 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.60000000000001 %
66.7 %
66.7 %
66.7 %
66.7 %
66.7 %
66.7 %
66.7 %
66.7 %
66.7 %
66.7 %
66.7 %
66.7 %
66.8 %
66.8 %
66.8 %
66.8 %
66.8 %
66.8 %
66.8 %
66.8 %
66.8 %
66.8 %
66.8 %
66.9 %
66.9 %
66.9 %
66.9 %
66.9 %
66.9 %
66.9 %
66.9 %
66.9 %
66.9 %
66.9 %
66.9 %
67.0 %
67.0 %
67.0 %
67.0 %
67.0 %
67.0 %
67.0 %
67.0 %
67.0 %
67.0 %
67.0 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.10000000000001 %
67.2 %
67.2 %
67.2 %
67.2 %
67.2 %
67.2 %
67.2 %
67.2 %
67.2 %
67.2 %
67.2 %
67

72.89999999999999 %
72.89999999999999 %
72.89999999999999 %
72.89999999999999 %
72.89999999999999 %
73.0 %
73.0 %
73.0 %
73.0 %
73.0 %
73.0 %
73.0 %
73.0 %
73.0 %
73.0 %
73.0 %
73.1 %
73.1 %
73.1 %
73.1 %
73.1 %
73.1 %
73.1 %
73.1 %
73.1 %
73.1 %
73.1 %
73.1 %
73.2 %
73.2 %
73.2 %
73.2 %
73.2 %
73.2 %
73.2 %
73.2 %
73.2 %
73.2 %
73.2 %
73.3 %
73.3 %
73.3 %
73.3 %
73.3 %
73.3 %
73.3 %
73.3 %
73.3 %
73.3 %
73.3 %
73.3 %
73.4 %
73.4 %
73.4 %
73.4 %
73.4 %
73.4 %
73.4 %
73.4 %
73.4 %
73.4 %
73.4 %
73.4 %
73.5 %
73.5 %
73.5 %
73.5 %
73.5 %
73.5 %
73.5 %
73.5 %
73.5 %
73.5 %
73.5 %
73.6 %
73.6 %
73.6 %
73.6 %
73.6 %
73.6 %
73.6 %
73.6 %
73.6 %
73.6 %
73.6 %
73.6 %
73.7 %
73.7 %
73.7 %
73.7 %
73.7 %
73.7 %
73.7 %
73.7 %
73.7 %
73.7 %
73.7 %
73.7 %
73.8 %
73.8 %
73.8 %
73.8 %
73.8 %
73.8 %
73.8 %
73.8 %
73.8 %
73.8 %
73.8 %
73.9 %
73.9 %
73.9 %
73.9 %
73.9 %
73.9 %
73.9 %
73.9 %
73.9 %
73.9 %
73.9 %
73.9 %
74.0 %
74.0 %
74.0 %
74.0 %
74.0 %
74.0 %
74.0 %
74.0 %
74.0 %
74.0 %
74.0 %
74.1 %
74.1

<ipython-input-9-660f05eed976>:15: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  sim_result_2 = pd.DataFrame(pd.np.empty((1, len(new_leads))),dtype = np.str)


In [ ]:
df['age']=df.apply(lambda x: x['age']+3,axis=1)

In [11]:
########################################### check 3 #######################################
sim_name =  pd.DataFrame(pd.np.empty((len(odoo_leads), len(new_leads)))) 
sim_name.columns = new_leads["Nom"]
sim_name.index = odoo_leads['Nom']

for i in range(0,sim_name.shape[0]):
    for j in range(0,sim_name.shape[1]):
        try:
            sim = SequenceMatcher(None,sim_name.index[i].lower(),sim_name.columns[j].lower()).ratio()
        except:
            sim = -1
        sim_name.iat[i,j] = sim
        
sim_result_3 = pd.DataFrame(pd.np.empty((1, len(new_leads))),dtype = np.str) 
sim_result_3.columns = new_leads["Entreprise"]

for j in range(0,sim_result_3.shape[1]):
    #sim_company_name_result.iat[0,j] = sim_company_name.loc[sim_company_name.iloc[:, j] == max(sim_company_name.iloc[:, j])].index[0]
    try:
        sim_result_3.iat[0,j] = sim_name[(sim_name.iloc[:, j] == max(sim_name.iloc[:, j])) & (sim_name.iloc[:, j] >0.9)].index[0]
    except:
        sim_result_3.iat[0,j] = "Pas de correspondances"

<ipython-input-11-d57384c8f6bc>:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  sim_name =  pd.DataFrame(pd.np.empty((len(odoo_leads), len(new_leads))))


KeyError: 'Nom'

In [ ]:
########################################### check 3bis #######################################
sim_name =  pd.DataFrame(pd.np.empty((len(odoo_leads), len(new_leads)))) 
sim_name.columns = new_leads["Nom"]
sim_name.index = odoo_leads['Prenom']

for i in range(0,sim_name.shape[0]):
    for j in range(0,sim_name.shape[1]):
        try:
            sim = SequenceMatcher(None,sim_name.index[i].lower(),sim_name.columns[j].lower()).ratio()
        except:
            sim = -1
        sim_name.iat[i,j] = sim
        
sim_result_3bis = pd.DataFrame(pd.np.empty((1, len(new_leads))),dtype = np.str) 
sim_result_3bis.columns = new_leads["Entreprise"]

for j in range(0,sim_result_3.shape[1]):
    #sim_company_name_result.iat[0,j] = sim_company_name.loc[sim_company_name.iloc[:, j] == max(sim_company_name.iloc[:, j])].index[0]
    try:
        sim_result_3bis.iat[0,j] = sim_name[(sim_name.iloc[:, j] == max(sim_name.iloc[:, j])) & (sim_name.iloc[:, j] >0.9)].index[0]
    except:
        sim_result_3bis.iat[0,j] = "Pas de correspondances"

In [ ]:
########################################### check 4 #######################################

sim_phone =  pd.DataFrame(pd.np.empty((len(odoo_leads), len(new_leads)))) 
sim_phone.columns = new_leads["id_tel"]
sim_phone.index = odoo_leads['id_tel']

for i in range(0,sim_phone.shape[0]):
    for j in range(0,sim_phone.shape[1]):
        try:
            sim = SequenceMatcher(None,sim_phone.index[i].lower(),sim_phone.columns[j].lower()).ratio()
        except:
            sim = -1
        sim_phone.iat[i,j] = sim
        
sim_result_4 = pd.DataFrame(pd.np.empty((1, len(new_leads))),dtype = np.str) 
sim_result_4.columns = new_leads["Entreprise"]

for j in range(0,sim_result_4.shape[1]):
    #sim_company_name_result.iat[0,j] = sim_company_name.loc[sim_company_name.iloc[:, j] == max(sim_company_name.iloc[:, j])].index[0]
    try:
        sim_result_4.iat[0,j] = sim_phone[(sim_phone.iloc[:, j] == max(sim_phone.iloc[:, j])) & (sim_phone.iloc[:, j] >0.95)].index[0]
    except:
        sim_result_4.iat[0,j] = "Pas de correspondances"

In [ ]:
sim_resultT = sim_result.T
sim_resultT.columns = ['check1']

sim_result_2T = sim_result_2.T
sim_result_2T.columns = ['check2']

sim_result_3T = sim_result_3.T
sim_result_3T.columns = ['check3']

sim_result_3bisT = sim_result_3bis.T
sim_result_3bisT.columns = ['check3bis']

sim_result_4T = sim_result_4.T
sim_result_4T.columns = ['check4']

In [ ]:
sim_resultT = sim_result.T
sim_resultT["check2"] = sim_result_2T["check2"]
sim_resultT["check3"] = sim_result_3T["check3"]
sim_resultT["check3bis"] = sim_result_3bisT["check3bis"]
sim_resultT["check4"] = sim_result_4T["check4"]

In [ ]:
sim_resultT.to_excel("C:/Users/grego/Desktop/i-lunch/Marketing/CRM/leads/check_new_leads.xlsx")

In [10]:
sim_result_2T = sim_result_2.T
sim_result_2T.columns = ['check2']

sim_result_2T.to_excel("C:/Users/grego/Desktop/i-lunch/Marketing/CRM/leads/check2_new_leads.xlsx")